In [64]:
# Importing libraries necessary for this project.

import requests
import pandas as pd
import html5lib
import time
from selenium import webdriver
from bs4 import BeautifulSoup
from datetime import date

In [65]:
# Create variables for the list of BAA & NBA Years needed.
# In this one, I'm getting all historical data.

baa_years = list(range(1947,1950))
nba_years = list(range(1950,2024))
total_years = baa_years + nba_years

print(total_years)
print("Total League Seasons:", len(total_years))

[1947, 1948, 1949, 1950, 1951, 1952, 1953, 1954, 1955, 1956, 1957, 1958, 1959, 1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]
Total League Seasons: 77


In [66]:
# Get the Basketball-Reference URLs to webscrape on.

baa_stats_url = "https://www.basketball-reference.com/leagues/BAA_{}_totals.html"
nba_stats_url = "https://www.basketball-reference.com/leagues/NBA_{}_totals.html"

#{} is where the years will be iterated on for each webpage

In [67]:
# Initialize the chromedriver to open an automated chrome window. For personal reasons, I have excluded my personal file path.

driver = webdriver.Chrome(executable_path="chromedriver_win32/chromedriver")

C:\Users\jayve\AppData\Local\Temp\ipykernel_14856\3341012693.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path="chromedriver_win32/chromedriver")


In [68]:
# Create a folder in your directory where the .html files will be saved on.
# We will be saving the pages on an html file to reduce website access
# and avoid getting banned for frequent access requests.

# For loop for getting BAA Season Totals
for year in baa_years:
    url = baa_stats_url.format(year)
    driver.get(url)
    driver.execute_script("window.scrollTo(1,10000)")
    # Used to scroll to bottom of webpage to load the whole table since it's being ran by JavaScript.
    # Not including this will only load a few rows of the table.
    time.sleep(5)
    
    html = driver.page_source
    with open("season stats websites/{}.html".format(year), "w+", encoding="utf-8") as f:
        f.write(html)
    print(str(year) + ".html saved!", end="\r")
    
print("Done! Data captured was from seasons",
      str(min(baa_years)-1)+"-"+str(min(baa_years)),
      "to",str(max(baa_years)-1)+"-"+str(max(baa_years)),
     )

Done! Data captured was from seasons 1946-1947 to 1948-1949


In [69]:
# Now we will do the same thing, but for the NBA Seasons

# For loop for getting NBA Season Totals
for year in nba_years:
    url = nba_stats_url.format(year)
    driver.get(url)
    driver.execute_script("window.scrollTo(1,10000)")
    # Used to scroll to bottom of webpage to load the whole table since it's being ran by JavaScript.
    # Not including this will only load a few rows of the table.
    time.sleep(5)
    
    html = driver.page_source
    with open("season stats websites/{}.html".format(year), "w+", encoding="utf-8") as f:
        f.write(html)
    print(str(year) + ".html saved!", end="\r")
    
print("Done! Data captured was from seasons",
      str(min(nba_years)-1)+"-"+str(min(nba_years)),
      "to",str(max(nba_years)-1)+"-"+str(max(nba_years)),
     )

Done! Data captured was from seasons 1949-1950 to 2022-2023


In [70]:
# From our saved .html files, we will now be using BeautifulSoup to gather data from the tables
# and append each year into one another before using Pandas to concat them.

df = []
for year in total_years:
    with open("season stats websites/{}.html".format(year), encoding="utf-8") as f:
        page = f.read()

    soup = BeautifulSoup(page, "html.parser")
    soup.find("tr", class_="thead").decompose()
    player_table = soup.find(id="totals_stats") 
    player = pd.read_html(str(player_table))[0]
    player['Year'] = year
    # This is for adding which year the stat was coming from.
    df.append(player)
    print("Progress:", year, end="\r")

print("Compiling done! Data captured was from seasons",
      str(min(total_years)-1)+"-"+str(min(total_years)),
      "to",str(max(total_years)-1)+"-"+str(max(total_years)),
      "Enjoy!")

Compiling done! Data captured was from seasons 1946-1947 to 2022-2023 Enjoy!


In [138]:
# Now let's use Pandas to concatenate our appended dataframes.

players = pd.concat(df)

In [139]:
# Quick check on the output.

players

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year
0,1,John Abramovic,F,27,PIT,47,NaN,NaN,202,834,...,NaN,NaN,NaN,35,NaN,NaN,NaN,161,527,1947
1,2,Chet Aubuchon,G,30,DTF,30,NaN,NaN,23,91,...,NaN,NaN,NaN,20,NaN,NaN,NaN,46,65,1947
2,3,Norm Baker,G,23,CHS,4,NaN,NaN,0,1,...,NaN,NaN,NaN,0,NaN,NaN,NaN,0,0,1947
3,4,Herschel Baltimore,F,25,STB,58,NaN,NaN,53,263,...,NaN,NaN,NaN,16,NaN,NaN,NaN,98,138,1947
4,5,John Barr,F,28,STB,58,NaN,NaN,124,438,...,NaN,NaN,NaN,54,NaN,NaN,NaN,164,295,1947
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
536,500,McKinley Wright IV,PG,24,DAL,18,1,180,17,42,...,6,19,25,30,7,3,12,17,41,2023
537,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,2023
538,501,Thaddeus Young,PF,34,TOR,44,9,712,98,175,...,64,84,148,68,49,5,36,79,219,2023
539,502,Trae Young,PG,24,ATL,48,48,1704,408,954,...,33,106,139,479,45,7,195,75,1286,2023


In [140]:
# Quick check if headers were also included.

players.loc[players["Player"] == "Player"]

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year
47,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,1947
70,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,1947
99,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,1947
122,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,1947
153,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,1947
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
451,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,2023
474,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,2023
495,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,2023
516,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,2023


In [141]:
# Now let's exclude those.

players = players.loc[players["Player"] != "Player"]

players

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year
0,1,John Abramovic,F,27,PIT,47,NaN,NaN,202,834,...,NaN,NaN,NaN,35,NaN,NaN,NaN,161,527,1947
1,2,Chet Aubuchon,G,30,DTF,30,NaN,NaN,23,91,...,NaN,NaN,NaN,20,NaN,NaN,NaN,46,65,1947
2,3,Norm Baker,G,23,CHS,4,NaN,NaN,0,1,...,NaN,NaN,NaN,0,NaN,NaN,NaN,0,0,1947
3,4,Herschel Baltimore,F,25,STB,58,NaN,NaN,53,263,...,NaN,NaN,NaN,16,NaN,NaN,NaN,98,138,1947
4,5,John Barr,F,28,STB,58,NaN,NaN,124,438,...,NaN,NaN,NaN,54,NaN,NaN,NaN,164,295,1947
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
535,499,Delon Wright,PG,30,WAS,25,2,550,53,128,...,23,57,80,92,48,7,20,36,146,2023
536,500,McKinley Wright IV,PG,24,DAL,18,1,180,17,42,...,6,19,25,30,7,3,12,17,41,2023
538,501,Thaddeus Young,PF,34,TOR,44,9,712,98,175,...,64,84,148,68,49,5,36,79,219,2023
539,502,Trae Young,PG,24,ATL,48,48,1704,408,954,...,33,106,139,479,45,7,195,75,1286,2023


In [142]:
# Then double check if they're still there.

players.loc[players["Player"] == "Player"]

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year


In [143]:
# Now let's also exclude the "Rk" column since this is irrelevant to our dataset.

players = players.drop(columns = 'Rk')

In [144]:
# Then reset your index just for standardization.
# .drop(columns = 'index') was included here because using .reset_index() retained the old index and made it into an additional column.

players = players.reset_index().drop(columns = 'index')

players # For checking your output at this stage of the table manipulation.

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year
0,John Abramovic,F,27,PIT,47,NaN,NaN,202,834,.242,...,NaN,NaN,NaN,35,NaN,NaN,NaN,161,527,1947
1,Chet Aubuchon,G,30,DTF,30,NaN,NaN,23,91,.253,...,NaN,NaN,NaN,20,NaN,NaN,NaN,46,65,1947
2,Norm Baker,G,23,CHS,4,NaN,NaN,0,1,.000,...,NaN,NaN,NaN,0,NaN,NaN,NaN,0,0,1947
3,Herschel Baltimore,F,25,STB,58,NaN,NaN,53,263,.202,...,NaN,NaN,NaN,16,NaN,NaN,NaN,98,138,1947
4,John Barr,F,28,STB,58,NaN,NaN,124,438,.283,...,NaN,NaN,NaN,54,NaN,NaN,NaN,164,295,1947
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29331,Delon Wright,PG,30,WAS,25,2,550,53,128,.414,...,23,57,80,92,48,7,20,36,146,2023
29332,McKinley Wright IV,PG,24,DAL,18,1,180,17,42,.405,...,6,19,25,30,7,3,12,17,41,2023
29333,Thaddeus Young,PF,34,TOR,44,9,712,98,175,.560,...,64,84,148,68,49,5,36,79,219,2023
29334,Trae Young,PG,24,ATL,48,48,1704,408,954,.428,...,33,106,139,479,45,7,195,75,1286,2023


In [149]:
# At this point, it is now up to you how you would want to manipulate the dataset further.
# If there are no further steps you want to take, let's now save your dataframe into a csv for your personal usecases.

today = date.today()

players.to_csv("datasets/nba_season_totals_{}.csv".format(today), index=False)